# Power Point Presentation maker

## Configs

In [2]:
from __future__ import annotations

import os

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    msg = "Missing API_KEY"
    raise OSError(msg)

## Pydantic class as OpenAI's expected response format

[OpenAI docs - structured format](https://platform.openai.com/docs/guides/structured-outputs)

In [ ]:
# class SingleSlide(BaseModel):
#     title: str
#     content: str
#     visual: str


# class ResponseFormat(BaseModel):
#     slide: list[SingleSlide]

In [ ]:
# from prompts import sys_prompt, usr_prompt

# client = OpenAI(api_key=api_key)
# completion = client.beta.chat.completions.parse(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": sys_prompt},
#         {"role": "user", "content": usr_prompt},
#     ],
#     response_format=ResponseFormat,
# )
# event = completion.choices[0].message.parsed

In [ ]:
class SingleSlide(BaseModel):
    Slide_Number: int
    Slide_Title: str
    Slide_Text: str
    Image_Prompt: str


class ResponseFormat(BaseModel):
    slide: list[SingleSlide]


class ResponseFormatBase(BaseModel):
    text: str

In [ ]:
def chat_completion(prompt: str, response_format):
    client = OpenAI(api_key=api_key)
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_format=response_format,
    )
    return completion.choices[0].message.parsed

In [ ]:
from typing import TypeVar

from pydantic import ValidationError

T = TypeVar("T", bound=BaseModel)


def validate_response(response: T | None, cls: type[T]) -> T:
    try:
        return cls.model_validate(response)
    except ValidationError as e:
        print(f"Error Validating response -> {response} \n {e}")
        raise

In [ ]:
from prompts import first_prompt, second_prompt

first_chat = validate_response(
    chat_completion(first_prompt, ResponseFormatBase), ResponseFormatBase,
)
second_chat = validate_response(
    chat_completion(second_prompt + "# Outline\n" + first_chat.text, ResponseFormat),
    ResponseFormat,
)

Error Validating response -> None 
 1 validation error for ResponseFormatBase
  Input should be a valid dictionary or instance of ResponseFormatBase [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type


ValidationError: 1 validation error for ResponseFormatBase
  Input should be a valid dictionary or instance of ResponseFormatBase [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

In [12]:
for i in second_chat.slide:
    print(f"{i.Slide_Number}->{i.Slide_Title}\n{i.Slide_Text}\n{i.Image_Prompt}\n")

1->Introduction to AI in Development
- AI is transforming software development by enhancing efficiency and creativity.  
- Practical AI tools help streamline processes for developers.  
- Objectives of this presentation:  
  - Introduce AI tools helpful for developers.  
  - Discuss real-world applications and benefits.
An illustrated representation of AI technology surrounding a software developer, symbolizing the integration of artificial intelligence in daily coding tasks.

2->Understanding AI Tools Available for Developers
- AI tools automate repetitive tasks, enabling developers to focus on complex issues.  
- Categories include:  
  - Code assistance  
  - Documentation tools  
  - Data analysis platforms  
- Benefits:  
  - Improved efficiency  
  - Enhanced productivity  
  - Spur innovation in coding practices.
Visual infographic showing various categories of AI tools relevant to developers, including icons for coding, documentation, and data analysis.

3->Code Assistance Tool

## Function to create images with Stable Diffusion

Using a local Stable Diffusion API to create the images, currently WebUI Forge.

In [ ]:
import base64
from pathlib import Path

import requests

# Define the URL and the payload to send.
url = "http://127.0.0.1:7860"


def imagecreator(prompt: str, name: str, width: int, height: int) -> None:
    """Access local Stable Diffusion to create images.

    Args:
        prompt (str): prompt for image creation
        name (str): Name of the resulting image
        width (int): Image Width
        height (int): Image Height

    """
    payload = {
        "prompt": prompt,
        "steps": 20,
        "sampler_name": "DPM++ 2M",
        "scheduler": "Beta",
        "cfg_scale": 1,
        "width": width,
        "height": height,
    }

    # Send said payload to said URL through the API.
    response = requests.post(url=f"{url}/sdapi/v1/txt2img", json=payload, timeout=300)
    r = response.json()

    # Decode and save the image.
    with Path(f"images/{name}.png").open("wb") as f:
        f.write(base64.b64decode(r["images"][0]))

## Create a PPTX with the responses generated with ChatGPT and the Stable Diffusion images

Using [Python-pptx](https://python-pptx.readthedocs.io/en/latest/index.html) for PowerPoint creation

In [ ]:
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
bullet_slide_layout = prs.slide_layouts[1]

for i in second_chat.slide:
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes

    # Add background image
    left = top = Inches(0)
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    imagecreator(i.visual + "NO TEXT", i.title, 955, 537)
    pic = slide.shapes.add_picture(
        str(Path(f"images/{i.title}.png")),
        left,
        top,
        slide_width,
        slide_height,
    )

    # Add Text
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]

    title_shape.text = i.title

    tf = body_shape.text_frame
    tf.text = i.content


prs.save("test.pptx")